In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
inspections = sc.textFile('nyc_restaurant_inspections.csv')

In [3]:
list(enumerate(inspections.first().split(',')))

[(0, 'CAMIS'),
 (1, 'DBA'),
 (2, 'BORO'),
 (3, 'BUILDING'),
 (4, 'STREET'),
 (5, 'ZIPCODE'),
 (6, 'PHONE'),
 (7, 'CUISINE DESCRIPTION'),
 (8, 'INSPECTION DATE'),
 (9, 'ACTION'),
 (10, 'VIOLATION CODE'),
 (11, 'VIOLATION DESCRIPTION'),
 (12, 'CRITICAL FLAG'),
 (13, 'SCORE'),
 (14, 'GRADE'),
 (15, 'GRADE DATE'),
 (16, 'RECORD DATE'),
 (17, 'INSPECTION TYPE')]

In [4]:
def extractDesc(partId, records):
    if partId==0:
        next(records)
    import csv
    reader = csv.reader(records)
    for row in reader:
        (camis,desc) = (int(row[0]), row[7])
        yield (camis, desc)

# each occurrence of a cami with its description
desc = inspections.mapPartitionsWithIndex(extractDesc)
desc.take(5)

[(50015075, 'Chicken'),
 (40364529, 'Jewish/Kosher'),
 (41580756, 'Caribbean'),
 (50001255, 'Italian'),
 (50016437, 'Korean')]

In [5]:
# group by cami, with the count
camis = desc.map(lambda x: (x[0],1)).reduceByKey(lambda x,y: x+y)
camis.take(5)

[(50015075, 12),
 (50001255, 13),
 (50015315, 50),
 (50008170, 30),
 (50000045, 29)]

In [6]:
# join cami and count with cami and description
# groupby cami to get unique values, and select only counts and descriptions
# set description as key, and set count to 1, as each cami is 1 restaraunt
# groupby description, adding counts for each description

desc_grouped = camis.join(desc).reduceByKey(lambda x,y:y).values().map(lambda x: (x[1], 1)).reduceByKey(lambda x,y: x+y)
desc_grouped.take(500)

[('American', 6002),
 ('Thai', 286),
 ('CafÃ©/Coffee/Tea', 1629),
 ('Polish', 27),
 ('Polynesian', 1),
 ('Chinese', 2399),
 ('Continental', 55),
 ('Peruvian', 74),
 ('French', 319),
 ('Fruits/Vegetables', 4),
 ('Sandwiches/Salads/Mixed Buffet', 253),
 ('Seafood', 181),
 ('Czech', 3),
 ('Delicatessen', 294),
 ('Juice, Smoothies, Fruit Salads', 382),
 ('Hamburgers', 378),
 ('Irish', 187),
 ('Bagels/Pretzels', 176),
 ('Barbecue', 50),
 ('Australian', 18),
 ('English', 16),
 ('Not Listed/Not Applicable', 11),
 ('Mexican', 877),
 ('Greek', 138),
 ('Vietnamese/Cambodian/Malaysia', 83),
 ('Indonesian', 9),
 ('Hotdogs', 32),
 ('Hawaiian', 32),
 ('Brazilian', 27),
 ('Soups', 5),
 ('Basque', 2),
 ('Bakery', 733),
 ('Steak', 85),
 ('Middle Eastern', 175),
 ('Pizza', 1186),
 ('Turkish', 62),
 ('Bangladeshi', 41),
 ('Armenian', 36),
 ('Nuts/Confectionary', 5),
 ('Californian', 6),
 ('Moroccan', 13),
 ('Iranian', 3),
 ('Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 840),
 ('Chic